## Standalone Single-Modality Pipelines (Cough-Only & Vowel-Only)
Complete standalone implementation training two separate LightGBM models, each using only one audio modality. No dependencies on previous cells.

In [2]:
# === STANDALONE SINGLE-MODALITY PIPELINES ===

import numpy as np
import pandas as pd
import json
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import xgboost as xgb

# Paths
ROOT = Path('.')
TRAIN_CSV = ROOT / 'train_air_respiratory.csv'
TEST_CSV = ROOT / 'test_air_respiratory.csv'
SOUNDS_DIR = ROOT / 'sounds'

# Helper to mean-pool JSON embeddings (returns None if missing or wrong shape)
def load_mean_embedding(json_path: Path):
    if not json_path.exists():
        return None
    with open(json_path, 'r') as f:
        data = json.load(f)
    arr = np.asarray(data, dtype=np.float32)
    if arr.ndim == 2:
        emb = arr.mean(axis=0)
    elif arr.ndim == 1:
        emb = arr
    else:
        return None
    return emb if emb.shape[0] == 512 else None

# Load and deduplicate train labels (one row per candidateID)
train_df = pd.read_csv(TRAIN_CSV).drop_duplicates(subset=['candidateID'])
label_map = dict(zip(train_df['candidateID'], train_df['disease']))

# === PIPELINE A: Cough-Only ===

print("=" * 60)
print("PIPELINE A: COUGH-ONLY")
print("=" * 60)

# Build cough-only dataset
cough_only_X, cough_only_y, cough_only_ids = [], [], []

for cid, label in label_map.items():
    base = SOUNDS_DIR / cid
    cough_emb = load_mean_embedding(base / 'emb_cough.json')
    if cough_emb is not None:
        cough_only_X.append(cough_emb)
        cough_only_y.append(label)
        cough_only_ids.append(cid)

cough_only_X = np.vstack(cough_only_X) if cough_only_X else np.empty((0, 512))
cough_only_y = np.array(cough_only_y)

print(f"Cough-only training samples: {len(cough_only_y)}")

# Train cough-only model with evaluation
XgBoost_CoughOnly = dict(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=400,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=3,
    n_jobs=4,
    random_state=42,
)

if len(cough_only_y) > 0:
    unique_classes = np.unique(cough_only_y)
    can_eval = len(cough_only_y) >= 10 and len(unique_classes) >= 2
    if can_eval:
        X_tr, X_val, y_tr, y_val = train_test_split(
            cough_only_X, cough_only_y, test_size=0.2, random_state=42, 
            stratify=cough_only_y if len(unique_classes) > 1 else None
        )
        cough_only_model = xgb.XGBClassifier(**XgBoost_CoughOnly)
        cough_only_model.fit(X_tr, y_tr)
        val_pred = cough_only_model.predict(X_val)
        print("\nCough-only validation metrics (holdout 20%):")
        print(f"  f1_macro: {f1_score(y_val, val_pred, average='macro'):.4f}")
        print(f"  accuracy: {accuracy_score(y_val, val_pred):.4f}")
        print(f"  precision_macro: {precision_score(y_val, val_pred, average='macro', zero_division=0):.4f}")
        print(f"  recall_macro: {recall_score(y_val, val_pred, average='macro'):.4f}")
    else:
        print("Cough-only: skipped holdout metrics (insufficient samples/classes)")

    # Refit on full data
    cough_only_model = xgb.XGBClassifier(**XgBoost_CoughOnly)
    cough_only_model.fit(cough_only_X, cough_only_y)
else:
    cough_only_model = None
    print("Cough-only: no data available")

# Inference for cough-only
test_df = pd.read_csv(TEST_CSV)
cough_preds = []

for cid in test_df['candidateID']:
    base = SOUNDS_DIR / cid
    cough_emb = load_mean_embedding(base / 'emb_cough.json')
    if cough_emb is not None and cough_only_model is not None:
        pred_class = cough_only_model.predict(cough_emb.reshape(1, -1))[0]
    else:
        pred_class = 2  # fallback
    cough_preds.append(pred_class)

cough_submission = pd.DataFrame({'candidateID': test_df['candidateID'], 'disease': cough_preds})
cough_submission_path = ROOT / 'submission_XgBoost_cough_only.csv'
cough_submission.to_csv(cough_submission_path, index=False)

print(f"\nSaved cough-only submission to {cough_submission_path}")
print(f"Total rows: {len(cough_submission)}")
print(f"Prediction distribution:\n{pd.Series(cough_preds).value_counts().sort_index()}")

PIPELINE A: COUGH-ONLY
Cough-only training samples: 515

Cough-only validation metrics (holdout 20%):
  f1_macro: 0.4281
  accuracy: 0.4951
  precision_macro: 0.4542
  recall_macro: 0.4362

Saved cough-only submission to submission_XgBoost_cough_only.csv
Total rows: 338
Prediction distribution:
0     75
1    117
2    146
Name: count, dtype: int64


In [ ]:
# === PIPELINE B: Vowel-Only ===

print("\n" + "=" * 60)
print("PIPELINE B: VOWEL-ONLY")
print("=" * 60)

# Build vowel-only dataset
vowel_only_X, vowel_only_y, vowel_only_ids = [], [], []

for cid, label in label_map.items():
    base = SOUNDS_DIR / cid
    vowel_emb = load_mean_embedding(base / 'emb_vowel.json')
    if vowel_emb is not None:
        vowel_only_X.append(vowel_emb)
        vowel_only_y.append(label)
        vowel_only_ids.append(cid)

vowel_only_X = np.vstack(vowel_only_X) if vowel_only_X else np.empty((0, 512))
vowel_only_y = np.array(vowel_only_y)

print(f"Vowel-only training samples: {len(vowel_only_y)}")

# Train vowel-only model with evaluation
XGb_VowelOnly = dict(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=400,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multiclass',
    metric='multi_logloss',
    num_class=3,
    n_jobs=4,
    random_state=42,
    verbose=-1
)

if len(vowel_only_y) > 0:
    unique_classes = np.unique(vowel_only_y)
    can_eval = len(vowel_only_y) >= 10 and len(unique_classes) >= 2
    if can_eval:
        X_tr, X_val, y_tr, y_val = train_test_split(
            vowel_only_X, vowel_only_y, test_size=0.2, random_state=42, 
            stratify=vowel_only_y if len(unique_classes) > 1 else None
        )
        vowel_only_model = xgb.XGBClassifier(**XGb_VowelOnly)
        vowel_only_model.fit(X_tr, y_tr)
        val_pred = vowel_only_model.predict(X_val)
        print("\nVowel-only validation metrics (holdout 20%):")
        print(f"  f1_macro: {f1_score(y_val, val_pred, average='macro'):.4f}")
        print(f"  accuracy: {accuracy_score(y_val, val_pred):.4f}")
        print(f"  precision_macro: {precision_score(y_val, val_pred, average='macro', zero_division=0):.4f}")
        print(f"  recall_macro: {recall_score(y_val, val_pred, average='macro'):.4f}")
    else:
        print("Vowel-only: skipped holdout metrics (insufficient samples/classes)")

    # Refit on full data
    vowel_only_model = xgb.XGBClassifier(**XGb_VowelOnly)
    vowel_only_model.fit(vowel_only_X, vowel_only_y)
else:
    vowel_only_model = None
    print("Vowel-only: no data available")

# Inference for vowel-only
vowel_preds = []

for cid in test_df['candidateID']:
    base = SOUNDS_DIR / cid
    vowel_emb = load_mean_embedding(base / 'emb_vowel.json')
    if vowel_emb is not None and vowel_only_model is not None:
        pred_class = vowel_only_model.predict(vowel_emb.reshape(1, -1))[0]
    else:
        pred_class = 2  # fallback
    vowel_preds.append(pred_class)

vowel_submission = pd.DataFrame({'candidateID': test_df['candidateID'], 'disease': vowel_preds})
vowel_submission_path = ROOT / 'submission_XgBoost_vowel_only.csv'
vowel_submission.to_csv(vowel_submission_path, index=False)

print(f"\nSaved vowel-only submission to {vowel_submission_path}")
print(f"Total rows: {len(vowel_submission)}")
print(f"Prediction distribution:\n{pd.Series(vowel_preds).value_counts().sort_index()}")

print("\n" + "=" * 60)
print("BOTH PIPELINES COMPLETED")
print("=" * 60)


PIPELINE B: VOWEL-ONLY
Vowel-only training samples: 540


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:10:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "metric", "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Vowel-only validation metrics (holdout 20%):
  f1_macro: 0.5134
  accuracy: 0.5741
  precision_macro: 0.5642
  recall_macro: 0.5228


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:10:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "metric", "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Saved vowel-only submission to submission_XgBoost_vowel_only.csv
Total rows: 338
Prediction distribution:
0     21
1    106
2    211
Name: count, dtype: int64

BOTH PIPELINES COMPLETED


: 